## Authenticate to Google Earth Engine

In [1]:
import ee
from pathlib import Path
import pandas as pd
import ast

ee.Authenticate() #Uncomment this whenever needed, once done usually not needed for 1-2 days
ee.Initialize(project='ee-raman')

In [2]:
def get_feature_collection(asset_id):
    """Check if an asset exists, and load it as a FeatureCollection if it does.
    Otherwise, return an empty FeatureCollection.
    
    Args:
        asset_id (str): The Earth Engine asset ID.
        
    Returns:
        ee.FeatureCollection: The loaded FeatureCollection or an empty one.
    """
    try:
        # Get asset information to check existence
        ee.data.getAsset(asset_id)
        print(f"Asset '{asset_id}' exists. Loading FeatureCollection.")
        return ee.FeatureCollection(asset_id)
    except Exception as e:
        print(f"Asset '{asset_id}' does not exist. Returning empty FeatureCollection.")
        return ee.FeatureCollection([])

In [3]:
import math
from itertools import product


# Function to convert latitude to pixel Y at a given zoom level
def lat_to_pixel_y(lat, zoom):
    sin_lat = math.sin(math.radians(lat))
    pixel_y = ((0.5 - math.log((1 + sin_lat) / (1 - sin_lat)) / (4 * math.pi)) * (2 ** (zoom + 8)))
    return pixel_y

# Function to convert longitude to pixel X at a given zoom level
def lon_to_pixel_x(lon, zoom):
    pixel_x = ((lon + 180) / 360) * (2 ** (zoom + 8))
    return pixel_x

# Function to convert pixel X to longitude
def pixel_x_to_lon(pixel_x, zoom):
    lon = (pixel_x / (2 ** (zoom + 8))) * 360 - 180
    return lon

# Function to convert pixel Y to latitude
def pixel_y_to_lat(pixel_y, zoom):
    n = math.pi - 2 * math.pi * pixel_y / (2 ** (zoom + 8))
    lat = math.degrees(math.atan(math.sinh(n)))
    return lat

def lat_lon_from_pixel(lat, lon, zoom, scale):
    """
    Given a starting latitude and longitude, calculate the latitude and longitude
    of the opposite corner of a 256x256 image at a given zoom level.
    """
    pixel_x = lon_to_pixel_x(lon, zoom)
    pixel_y = lat_to_pixel_y(lat, zoom)
    
    new_lon = pixel_x_to_lon(pixel_x + 256*scale, zoom)
    new_lat = pixel_y_to_lat(pixel_y + 256*scale, zoom)

    return new_lat, new_lon

"""

Helper function for dividing an roi into blocks

"""
def get_n_boxes(lat, lon, n, zoom, scale):
    diagonal_lat_lon = [(lat, lon),]
    for i in range(n):
        new_lat_lon = lat_lon_from_pixel(lat, lon, zoom, scale)
        diagonal_lat_lon.append(new_lat_lon)
        lat, lon = new_lat_lon
    lats = [i[0] for i in diagonal_lat_lon]
    longs = [i[1] for i in diagonal_lat_lon]
    return list(product(lats, longs))

def get_points(roi, directory):
    points_file = Path(directory + "/status.csv")
    if points_file.is_file():
        df = pd.read_csv(directory + "/status.csv", index_col=False)
        df["points"] = df['points'].apply(ast.literal_eval)
        return df
    zoom = 17
    scale = 16
    bounds = roi.bounds().coordinates().get(0).getInfo()
    lons = sorted([i[0] for i in bounds])
    lats = sorted([i[1] for i in bounds])
    starting_point = lats[-1], lons[0]
    min_, max_ = (
        [lon_to_pixel_x(lons[0], zoom), lat_to_pixel_y(lats[0], zoom) ],
        [lon_to_pixel_x(lons[-1], zoom), lat_to_pixel_y(lats[-1], zoom)]
        )
    iterations = math.ceil(max(abs(min_[0] -  max_[0]), abs(min_[1] - max_[1]))/256/16)
    points = get_n_boxes(starting_point[0], starting_point[1], iterations, zoom, scale)
    intersect_list = []
    print(len(points))
    index = 0
    for point in points:
        top_left = point
        bottom_right = lat_lon_from_pixel(top_left[0], top_left[1], zoom, scale)
        rectangle = ee.Geometry.Rectangle([(top_left[1], top_left[0]), (bottom_right[1], bottom_right[0])])
        print(top_left, bottom_right)
        intersects = roi.geometry().intersects(rectangle, ee.ErrorMargin(1)).getInfo()
        if intersects:
            intersect_list.append((index, (top_left,bottom_right)))
            index+=1
        print(intersects)
    df = pd.DataFrame(intersect_list, columns=["index", "points"])
    df["overall_status"] = False
    df["download_status"] = False
    df["model_status"] = False
    df["segmentation_status"] = False
    df["postprocessing_status"] = False
    df["plantation_status"] = False
    df.to_csv(directory + "/status.csv", index=False)
    return df

roi_boundary = ee.FeatureCollection("users/mtpictd/india_block_boundaries").filter(ee.Filter.eq("block", "Bheramgarh"))
directory = "Area_bheramgarh"
roi_boundary = ee.FeatureCollection("users/mtpictd/india_block_boundaries").filter(ee.Filter.eq("block", "Jamkhed"))
directory = "Area_jamkhed"
roi_boundary = ee.FeatureCollection("users/mtpictd/india_block_boundaries").filter(ee.Filter.eq("block", "Peddapally"))
directory = "Area_Peddapally"

blocks_df = get_points(roi_boundary, directory)
points = list(blocks_df["points"])

roi_boundary = ee.FeatureCollection([ee.Feature(ee.Geometry.Rectangle([top_left[1], bottom_right[0], bottom_right[1], top_left[0]])) for top_left, bottom_right in points])


In [4]:


mapping = {
    "farm": 1,
    "plantation": 2,
    "scrubland": 3,
    "rest": 0
}
reversed_mapping = {v: k for k, v in mapping.items()}
reversed_ee_mapping = ee.Dictionary(reversed_mapping)

easy_farm = [
    ee.Filter.gte("rect", 0.67),
    ee.Filter.gt("size", 500),
    ee.Filter.lt("size", 2000),
    ee.Filter.lt("ent", 1)
    ]
easy_scurbland = [
    ee.Filter.gte("size", 60000),
    ee.Filter.gt("red", 0.9)
    ]
easy_plantation = [
    ee.Filter.lt("area", 20000),
    ee.Filter.gt("area", 1000)
]


all = get_feature_collection("projects/ee-raman/assets/" + directory + "_boundaries")
farm = all.filter(ee.Filter.And(*easy_farm))
scrubland = all.filter(ee.Filter.And(easy_scurbland))
plantation = all.filter(ee.Filter.eq("class", "plantation")).map(lambda x: x.set("area", x.geometry().area())).filter(ee.Filter.And(easy_plantation))

farm_buffer = farm.map(lambda x: x.buffer(10))
farm_image = ee.Image(0)
farm_mask = farm_image.clip(farm_buffer).mask()
farm_vectors = farm_mask.toInt().reduceToVectors(
    geometry=roi_boundary,
    scale=10,  # Change based on your resolution
    geometryType='polygon',
    labelProperty='zone',
    reducer=ee.Reducer.countEvery()
)
farm_vectors = farm_vectors.filter(ee.Filter.eq('zone', 1)).map(lambda x: x.set("count", farm.filterBounds(x.geometry()).size())).filter(ee.Filter.gt('count', 3))
#farm = farm.filterBounds(farm_vectors)

label_image = ee.Image(0).rename("label")
farm_mask = label_image.clip(farm).mask()
scrubland_mask = label_image.clip(scrubland).mask()
plantation_mask = label_image.clip(plantation).mask()

label_image = label_image.where(farm_mask, mapping["farm"]).where(scrubland_mask, mapping["scrubland"]).where(plantation_mask, mapping["plantation"])
ts_data = ee.Image("projects/ee-raman/assets/ts_data_" + directory)

# Classes to sample (exclude background = 0)
class_values = [(1,20000), (2,20000), (3,20000)]

# Empty list to store samples
samples_list = []

for class_val, points in class_values:
    # Create a mask for the class
    class_mask = label_image.eq(class_val)
    
    # Mask the ts_image to only include pixels of this class
    masked_ts = ts_data.updateMask(class_mask)
    
    # Sample uniformly from the masked image
    class_samples = masked_ts.addBands(label_image.rename('class')) \
        .stratifiedSample(
            numPoints=points,  # adjust as needed
            classBand='class',
            classValues=[class_val],
            classPoints=[points],  # adjust per class
            scale=10,
            region=ts_data.geometry(),
            seed=42,
            geometries=True
        )
    
    samples_list.append(class_samples)

all_samples = samples_list[0].merge(samples_list[1]).merge(samples_list[2])

Asset 'projects/ee-raman/assets/Area_Peddapally_boundaries' exists. Loading FeatureCollection.


In [5]:
classifier = ee.Classifier.smileRandomForest(50).train(
    features=all_samples,
    classProperty='class',
    inputProperties=ts_data.bandNames()
)
classified = ts_data.classify(classifier)

In [6]:
def assign_mode_label(feature):
    class_values = classified.reduceRegion(
        reducer=ee.Reducer.mode(),
        geometry=feature.geometry(),
        scale=30,  # Adjust scale as per resolution
        bestEffort=True
    )
    return feature.set('class', class_values.get('classification'))

# Apply function to test features
all_labels = all.map(assign_mode_label).filter(ee.Filter.notNull(['class']))
all_labels = all_labels.map(lambda x: x.set('class', reversed_ee_mapping.get(ee.Number(x.get("class")).int())))

In [6]:
task = ee.batch.Export.table.toAsset(
    collection=all_labels,
    description='Classification',
    assetId="projects/ee-raman/assets/" + directory + "_boundaries_refined"
)

# Start the task
task.start()

In [ ]:
import geemap



final_lulc_img = ee.Image("projects/ee-raman/assets/Area_Peddapally_v4")
#final_lulc_img_corrected = ee.Image("projects/ee-raman/assets/" + filename_prefix + "_corrected_slide")
#all = get_feature_collection("projects/ee-raman/assets/all_" + suffix)
#farm = all.filter(ee.Filter.eq("class", "farm"))
#scrubland = all.filter(ee.Filter.eq("class", "scrubland"))
#plantation = all.filter(ee.Filter.eq("class", "plantation"))

m = geemap.Map()
m.centerObject(roi_boundary)


url = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}'
m.layout.height = '1000px'
m.add_tile_layer(url, name="Google Map", attribution="Google")
#m.add_basemap(, )
m.addLayerControl()
#m.addLayer(fields_boundary, {}, 'Fields')

palette = [
    '303000', '000000', '000000', '000000', '000000', 'f75cff', 
    '991695', 'e6ab2c', '3bef34', '2baa25', 'e6ef34', 'a1a524', 'eea15e'
]

palette_corrected = [
    '303000', '000000', '000000', '000000', '000000', 'f75cff', 
    '991695', 'e6ab2c', '3bef34', '2baa25', 'e6ef34', 'a1a524', 'eea15e', '29554E'
]

vis_params = {
    'min': 0,
    'max': 12,
    'palette': palette
}

vis_params_corrected = {
    'min': 0,
    'max': 13,
    'palette': palette_corrected
}

labels = """0. Background
1. Built-up
2. Water (Kharif)
3. Water (Kharif + Rabi)
4. Water (Kharif + Rabi + Zaid)
5. Croplands
6. Tree/Forest
7. Barren Land
8. Single Kharif (Light blue)
9. Single Non-Kharif
10. Double (Dark-Blue)
11. Triple
12. Shrub_Scrub
13. Plantation"""
labels = [i.split(". ")[-1] for i in labels.split("\n")]

palette_ = ['3bef34', '991695', '0000FF']
vis_params_ = {
    'min': 1,
    'max': 3,
    'palette': palette_
}


m.addLayer(final_lulc_img.select("predicted_label"), vis_params, 'LULC')
#m.addLayer(final_lulc_img_corrected.select("predicted_label"), vis_params_corrected, 'LULC_corrected')
#m.addLayer(classified, vis_params_, 'predictions')

m.add_legend(keys=labels, colors=palette_corrected, position="bottomleft")
#m.addLayer(classified.select("classification"), vis_params_, "rest_with_labels" )
m.addLayer(farm_vectors, {}, 'farm_vectors')
m.addLayer(farm, {}, 'farm')

m.addLayer(scrubland, {}, 'scrubland')
m.addLayer(all, {}, 'all')
m.addLayer(plantation.map(lambda x: x.set("area", x.geometry().area())).filter(ee.Filter.lt("area", 20000)).filter(ee.Filter.gt("area", 1000)), {}, 'plantation')
#m.addLayer(rest, {}, 'rest')
#m.addLayer(rest_all.filter(ee.Filter.eq("predicted_class", 0)), {}, 'predicted_farm')
#m.addLayer(rest_all.filter(ee.Filter.eq("predicted_class", 1)), {}, 'predicted_scrubland')
#m.addLayer(rest_all.filter(ee.Filter.eq("predicted_class", 2)), {}, 'predicted_plantation')
#m.addLayer(aatif_plantation, {}, 'aatif_plantation')


m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [ ]:
directory = "Area_bheramgarh"
directory = "Area_jamkhed"
directory = "Area_peddapally"

suffix = directory.split("_")[-1]
mapping = {
    "farm": 1,
    "plantation": 2,
    "scrubland": 3,
    "rest": 0
}
reversed_mapping = {v: k for k, v in mapping.items()}
reversed_ee_mapping = ee.Dictionary(reversed_mapping)

def get_class_and_mapping(vectors, vector_name):
    vector_of_class = vectors.filter(ee.Filter.eq('class', vector_name))
    vector_of_class_with_map = vector_of_class.map(lambda f: f.set('class', mapping[vector_name]))
    return vector_of_class, vector_of_class_with_map

all = get_feature_collection("projects/ee-raman/assets/" + directory + "_boundaries")
farm, farm_with_map = get_class_and_mapping(all, "farm")
scrubland, scrubland_with_map = get_class_and_mapping(all, "scrubland")
plantation, plantation_with_map = get_class_and_mapping(all, "plantation")
rest, rest_with_map = get_class_and_mapping(all, "rest")

plantation = plantation.map(lambda x: x.set("area", x.geometry().area())).filter(ee.Filter.lt("area", 20000)).filter(ee.Filter.gt("area", 1000))

label_image = ee.Image(0).rename("label")
farm_mask = label_image.clip(farm).mask()
scrubland_mask = label_image.clip(scrubland).mask()
plantation_mask = label_image.clip(plantation).mask()

label_image = label_image.where(farm_mask, mapping["farm"]).where(scrubland_mask, mapping["scrubland"]).where(plantation_mask, mapping["plantation"])
ts_data = ee.Image("projects/ee-raman/assets/ts_data_" + directory)

# Classes to sample (exclude background = 0)
class_values = [(1,20000), (2,20000), (3,20000)]

# Empty list to store samples
samples_list = []

for class_val, points in class_values:
    # Create a mask for the class
    class_mask = label_image.eq(class_val)
    
    # Mask the ts_image to only include pixels of this class
    masked_ts = ts_data.updateMask(class_mask)
    
    # Sample uniformly from the masked image
    class_samples = masked_ts.addBands(label_image.rename('class')) \
        .stratifiedSample(
            numPoints=points,  # adjust as needed
            classBand='class',
            classValues=[class_val],
            classPoints=[points],  # adjust per class
            scale=10,
            region=ts_data.geometry(),
            seed=42,
            geometries=True
        )
    
    samples_list.append(class_samples)

all_samples = samples_list[0].merge(samples_list[1]).merge(samples_list[2])

#training_features = farm_with_map.merge(scrubland_with_map).merge(plantation_with_map)
